In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import psycopg2

In [2]:
# Make a reference to the books.csv file path
csv_path = "Data/annual_conc_by_monitor_2019.csv"

# Import the books.csv file as a DataFrame
air_df = pd.read_csv(csv_path, encoding="utf-8")
air_df

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,75th Percentile,50th Percentile,10th Percentile,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
0,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,1 HOUR,...,0.054,0.045,0.029,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2020-02-18
1,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2020-02-18
2,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2020-02-18
3,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2020-02-18
4,1,3,10,88101,1,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,24 HOUR,...,9.500,7.300,3.800,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2020-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62949,80,2,24,85101,1,32.654420,-115.407267,NAD83,PM10 - LC,24 HOUR,...,55.000,38.000,18.000,NaN,Calzada Cetys,Country Of Mexico,BAJA CALIFORNIA NORTE,Mexicali,NaN,2020-04-03
62950,80,26,8012,44201,1,32.466389,-114.768611,WGS84,Ozone,1 HOUR,...,0.057,0.051,0.038,San Luis Rio Colorado Well 10,Calle 15 & Venustiano Carranza Water Well #10,Country Of Mexico,SONORA,NaN,NaN,2020-01-17
62951,80,26,8012,44201,1,32.466389,-114.768611,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.053,0.046,0.032,San Luis Rio Colorado Well 10,Calle 15 & Venustiano Carranza Water Well #10,Country Of Mexico,SONORA,NaN,NaN,2020-01-17
62952,80,26,8012,44201,1,32.466389,-114.768611,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.053,0.046,0.032,San Luis Rio Colorado Well 10,Calle 15 & Venustiano Carranza Water Well #10,Country Of Mexico,SONORA,NaN,NaN,2020-01-17


In [3]:
# Remove unecessary columns from the DataFrame and save the new DataFrame
base_df = air_df[["State Code", "County Code", "Site Num", "State Name", "City Name", "POC", "Latitude", "Longitude", "Parameter Name", "Arithmetic Mean", "Pollutant Standard"]]
base_df.head()

,State Code,County Code,Site Num,State Name,City Name,POC,Latitude,Longitude,Parameter Name,Arithmetic Mean,Pollutant Standard
0,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.044808,Ozone 1-hour 1979
1,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 1997
2,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 2008
3,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040489,Ozone 8-hour 2015
4,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 24-hour 2006


In [4]:
#Rename columns
base_df = base_df.rename(columns={
    "State Name": "state", 
    "City Name": "city",
    "Latitude": "latitude",
    "Longitude": "longitude",
    "Parameter Name": "pollutant",
    "Arithmetic Mean": "average"
})

#Set index to....city?
#base_df.set_index("City", inplace=True)

base_df.head()

,State Code,County Code,Site Num,state,city,POC,latitude,longitude,pollutant,average,Pollutant Standard
0,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.044808,Ozone 1-hour 1979
1,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 1997
2,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 2008
3,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040489,Ozone 8-hour 2015
4,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 24-hour 2006


## OZONE DATA:

In [5]:
# Ozone df
ozone = base_df.loc[base_df['pollutant'] == 'Ozone']
ozone.head()

,State Code,County Code,Site Num,state,city,POC,latitude,longitude,pollutant,average,Pollutant Standard
0,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.044808,Ozone 1-hour 1979
1,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 1997
2,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 2008
3,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040489,Ozone 8-hour 2015
12,1,33,1002,Alabama,Muscle Shoals,1,34.762619,-87.638097,Ozone,0.043192,Ozone 1-hour 1979


In [6]:
# Choosing to use only one pollutant standard (I chose the most recent which is Ozone 8hr 2015)
# I did this so each city (or coordinates) will have one of each test data
ozone = ozone.loc[ozone['Pollutant Standard'] == 'Ozone 8-hour 2015']
ozone

,State Code,County Code,Site Num,state,city,POC,latitude,longitude,pollutant,average,Pollutant Standard
3,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040489,Ozone 8-hour 2015
15,1,33,1002,Alabama,Muscle Shoals,1,34.762619,-87.638097,Ozone,0.038221,Ozone 8-hour 2015
27,1,49,9991,Alabama,Crossville,1,34.289001,-85.970065,Ozone,0.044582,Ozone 8-hour 2015
31,1,51,4,Alabama,Wetumpka,1,32.535680,-86.255193,Ozone,0.040902,Ozone 8-hour 2015
46,1,55,11,Alabama,Southside,1,33.904039,-86.053867,Ozone,0.044016,Ozone 8-hour 2015
...,...,...,...,...,...,...,...,...,...,...,...
62653,56,45,3,Wyoming,NaN,1,43.873056,-104.191944,Ozone,0.045297,Ozone 8-hour 2015
62702,72,21,10,Puerto Rico,Bayamon,1,18.420089,-66.150615,Ozone,0.013117,Ozone 8-hour 2015
62816,72,33,8,Puerto Rico,Catano,1,18.440774,-66.126531,Ozone,0.009864,Ozone 8-hour 2015
62858,72,97,7,Puerto Rico,Mayaguez,1,18.216038,-67.144107,Ozone,0.005862,Ozone 8-hour 2015


In [7]:
# Search for duplicates
ozone.duplicated()

3        False
15       False
27       False
31       False
46       False
         ...  
62653    False
62702    False
62816    False
62858    False
62953    False
Length: 1304, dtype: bool

In [8]:
# I run this code anyway to see if any duplicates were missed or hidden
ozone = ozone.drop_duplicates()
ozone

,State Code,County Code,Site Num,state,city,POC,latitude,longitude,pollutant,average,Pollutant Standard
3,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040489,Ozone 8-hour 2015
15,1,33,1002,Alabama,Muscle Shoals,1,34.762619,-87.638097,Ozone,0.038221,Ozone 8-hour 2015
27,1,49,9991,Alabama,Crossville,1,34.289001,-85.970065,Ozone,0.044582,Ozone 8-hour 2015
31,1,51,4,Alabama,Wetumpka,1,32.535680,-86.255193,Ozone,0.040902,Ozone 8-hour 2015
46,1,55,11,Alabama,Southside,1,33.904039,-86.053867,Ozone,0.044016,Ozone 8-hour 2015
...,...,...,...,...,...,...,...,...,...,...,...
62653,56,45,3,Wyoming,NaN,1,43.873056,-104.191944,Ozone,0.045297,Ozone 8-hour 2015
62702,72,21,10,Puerto Rico,Bayamon,1,18.420089,-66.150615,Ozone,0.013117,Ozone 8-hour 2015
62816,72,33,8,Puerto Rico,Catano,1,18.440774,-66.126531,Ozone,0.009864,Ozone 8-hour 2015
62858,72,97,7,Puerto Rico,Mayaguez,1,18.216038,-67.144107,Ozone,0.005862,Ozone 8-hour 2015


In [9]:
# Search for nan values
ozone.isna().any()

State Code            False
County Code           False
Site Num              False
state                 False
city                   True
POC                   False
latitude              False
longitude             False
pollutant             False
average               False
Pollutant Standard    False
dtype: bool

In [10]:
# Remove POC and Pollutant Standard columns from the Data frame
ozone = ozone[["State Code", "County Code", "Site Num","state", "city", "latitude", "longitude", "pollutant", "average"]]
ozone.head()

,State Code,County Code,Site Num,state,city,latitude,longitude,pollutant,average
3,1,3,10,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.040489
15,1,33,1002,Alabama,Muscle Shoals,34.762619,-87.638097,Ozone,0.038221
27,1,49,9991,Alabama,Crossville,34.289001,-85.970065,Ozone,0.044582
31,1,51,4,Alabama,Wetumpka,32.535680,-86.255193,Ozone,0.040902
46,1,55,11,Alabama,Southside,33.904039,-86.053867,Ozone,0.044016


## PM.2.5 DATA

In [11]:
# working off the base
base_df.head()

,State Code,County Code,Site Num,state,city,POC,latitude,longitude,pollutant,average,Pollutant Standard
0,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.044808,Ozone 1-hour 1979
1,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 1997
2,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 2008
3,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,Ozone,0.040489,Ozone 8-hour 2015
4,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 24-hour 2006


In [12]:
# Ozone df
pm = base_df.loc[base_df['pollutant'] == 'PM2.5 - Local Conditions']
pm.head()

,State Code,County Code,Site Num,state,city,POC,latitude,longitude,pollutant,average,Pollutant Standard
4,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 24-hour 2006
5,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 24-hour 2012
6,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 Annual 2006
7,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 Annual 2012
8,1,27,1,Alabama,Ashland,1,33.284928,-85.803608,PM2.5 - Local Conditions,7.550467,PM25 24-hour 2006


In [13]:
# Choosing to use only one pollutant standard (I chose the most recent which is PM2.5 Annual 2012)
# I did this so each city (or coordinates) will have one of each test data
pm = pm.loc[pm['Pollutant Standard'] == 'PM25 Annual 2012']
pm.head(20)

,State Code,County Code,Site Num,state,city,POC,latitude,longitude,pollutant,average,Pollutant Standard
7,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 Annual 2012
11,1,27,1,Alabama,Ashland,1,33.284928,-85.803608,PM2.5 - Local Conditions,7.550467,PM25 Annual 2012
19,1,33,1002,Alabama,Muscle Shoals,1,34.762619,-87.638097,PM2.5 - Local Conditions,7.213235,PM25 Annual 2012
23,1,49,1003,Alabama,Crossville,1,34.288567,-85.969858,PM2.5 - Local Conditions,7.550000,PM25 Annual 2012
42,1,55,10,Alabama,Gadsden,1,33.991494,-85.992647,PM2.5 - Local Conditions,8.410526,PM25 Annual 2012
50,1,69,3,Alabama,Dothan,1,31.224783,-85.390789,PM2.5 - Local Conditions,8.115652,PM25 Annual 2012
118,1,73,23,Alabama,Birmingham,1,33.553056,-86.815000,PM2.5 - Local Conditions,10.310833,PM25 Annual 2012
122,1,73,23,Alabama,Birmingham,2,33.553056,-86.815000,PM2.5 - Local Conditions,10.100000,PM25 Annual 2012
127,1,73,23,Alabama,Birmingham,3,33.553056,-86.815000,PM2.5 - Local Conditions,10.552222,PM25 Annual 2012
262,1,73,1005,Alabama,NaN,1,33.331111,-87.003611,PM2.5 - Local Conditions,8.701667,PM25 Annual 2012


In [14]:
# Get rid of duplicates using POC=1 (ex, index 323&327, 512&516 are same)
pm = pm.loc[pm['POC'] == 1]
pm.head(20)

,State Code,County Code,Site Num,state,city,POC,latitude,longitude,pollutant,average,Pollutant Standard
7,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 Annual 2012
11,1,27,1,Alabama,Ashland,1,33.284928,-85.803608,PM2.5 - Local Conditions,7.550467,PM25 Annual 2012
19,1,33,1002,Alabama,Muscle Shoals,1,34.762619,-87.638097,PM2.5 - Local Conditions,7.213235,PM25 Annual 2012
23,1,49,1003,Alabama,Crossville,1,34.288567,-85.969858,PM2.5 - Local Conditions,7.550000,PM25 Annual 2012
42,1,55,10,Alabama,Gadsden,1,33.991494,-85.992647,PM2.5 - Local Conditions,8.410526,PM25 Annual 2012
50,1,69,3,Alabama,Dothan,1,31.224783,-85.390789,PM2.5 - Local Conditions,8.115652,PM25 Annual 2012
118,1,73,23,Alabama,Birmingham,1,33.553056,-86.815000,PM2.5 - Local Conditions,10.310833,PM25 Annual 2012
262,1,73,1005,Alabama,NaN,1,33.331111,-87.003611,PM2.5 - Local Conditions,8.701667,PM25 Annual 2012
283,1,73,1010,Alabama,Leeds,1,33.545278,-86.549167,PM2.5 - Local Conditions,8.363934,PM25 Annual 2012
323,1,73,2003,Alabama,Birmingham,1,33.499722,-86.924167,PM2.5 - Local Conditions,8.705000,PM25 Annual 2012


In [15]:
# There are duplicates in Alaska Fairbanks, index 855 & 856. 
pm = pm.drop_duplicates()
pm.head(30)

,State Code,County Code,Site Num,state,city,POC,latitude,longitude,pollutant,average,Pollutant Standard
7,1,3,10,Alabama,Fairhope,1,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 Annual 2012
11,1,27,1,Alabama,Ashland,1,33.284928,-85.803608,PM2.5 - Local Conditions,7.550467,PM25 Annual 2012
19,1,33,1002,Alabama,Muscle Shoals,1,34.762619,-87.638097,PM2.5 - Local Conditions,7.213235,PM25 Annual 2012
23,1,49,1003,Alabama,Crossville,1,34.288567,-85.969858,PM2.5 - Local Conditions,7.550000,PM25 Annual 2012
42,1,55,10,Alabama,Gadsden,1,33.991494,-85.992647,PM2.5 - Local Conditions,8.410526,PM25 Annual 2012
50,1,69,3,Alabama,Dothan,1,31.224783,-85.390789,PM2.5 - Local Conditions,8.115652,PM25 Annual 2012
118,1,73,23,Alabama,Birmingham,1,33.553056,-86.815000,PM2.5 - Local Conditions,10.310833,PM25 Annual 2012
262,1,73,1005,Alabama,NaN,1,33.331111,-87.003611,PM2.5 - Local Conditions,8.701667,PM25 Annual 2012
283,1,73,1010,Alabama,Leeds,1,33.545278,-86.549167,PM2.5 - Local Conditions,8.363934,PM25 Annual 2012
323,1,73,2003,Alabama,Birmingham,1,33.499722,-86.924167,PM2.5 - Local Conditions,8.705000,PM25 Annual 2012


In [16]:
# Alaska Fairbanks! I am not sure how to get rid of those duplicated in Fairbanks... I will leave them there

In [17]:
# Remove POC and Pollutant Standard columns from the Data frame
pm = pm[["State Code", "County Code", "Site Num", "state", "city", "latitude", "longitude", "pollutant", "average"]]
pm.head()

,State Code,County Code,Site Num,state,city,latitude,longitude,pollutant,average
7,1,3,10,Alabama,Fairhope,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402
11,1,27,1,Alabama,Ashland,33.284928,-85.803608,PM2.5 - Local Conditions,7.550467
19,1,33,1002,Alabama,Muscle Shoals,34.762619,-87.638097,PM2.5 - Local Conditions,7.213235
23,1,49,1003,Alabama,Crossville,34.288567,-85.969858,PM2.5 - Local Conditions,7.550000
42,1,55,10,Alabama,Gadsden,33.991494,-85.992647,PM2.5 - Local Conditions,8.410526


In [18]:
# Search for duplicates
pm.duplicated()

7        False
11       False
19       False
23       False
42       False
         ...  
62823    False
62835    False
62844    False
62871    False
62929    False
Length: 667, dtype: bool

In [19]:
# Search for nan values
pm.isna().any()

State Code     False
County Code    False
Site Num       False
state          False
city            True
latitude       False
longitude      False
pollutant      False
average        False
dtype: bool

## Merge ozone and pm df

In [20]:
merge_df = pd.merge(ozone, pm, on=["State Code", "County Code", "Site Num", "state", "city", "latitude", "longitude"], how="outer")
merge_df

,State Code,County Code,Site Num,state,city,latitude,longitude,pollutant_x,average_x,pollutant_y,average_y
0,1,3,10,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.040489,PM2.5 - Local Conditions,7.551402
1,1,33,1002,Alabama,Muscle Shoals,34.762619,-87.638097,Ozone,0.038221,PM2.5 - Local Conditions,7.213235
2,1,49,9991,Alabama,Crossville,34.289001,-85.970065,Ozone,0.044582,NaN,NaN
3,1,51,4,Alabama,Wetumpka,32.535680,-86.255193,Ozone,0.040902,NaN,NaN
4,1,55,11,Alabama,Southside,33.904039,-86.053867,Ozone,0.044016,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1639,72,53,3,Puerto Rico,NaN,18.383983,-65.618888,NaN,NaN,PM2.5 - Local Conditions,6.868067
1640,72,57,12,Puerto Rico,Guayama,17.955378,-66.162122,NaN,NaN,PM2.5 - Local Conditions,5.303571
1641,72,61,5,Puerto Rico,Guaynabo,18.440095,-66.114460,NaN,NaN,PM2.5 - Local Conditions,7.487179
1642,72,113,4,Puerto Rico,Ponce,18.009558,-66.627249,NaN,NaN,PM2.5 - Local Conditions,6.285841


In [21]:
#Rename columns
merge_df = merge_df.rename(columns={
    "pollutant_x": "ozone", 
    "average_x": "mean_ozone",
    "pollutant_y": "pm25",
    "average_y": "mean_pm25"
})
merge_df

,State Code,County Code,Site Num,state,city,latitude,longitude,ozone,mean_ozone,pm25,mean_pm25
0,1,3,10,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.040489,PM2.5 - Local Conditions,7.551402
1,1,33,1002,Alabama,Muscle Shoals,34.762619,-87.638097,Ozone,0.038221,PM2.5 - Local Conditions,7.213235
2,1,49,9991,Alabama,Crossville,34.289001,-85.970065,Ozone,0.044582,NaN,NaN
3,1,51,4,Alabama,Wetumpka,32.535680,-86.255193,Ozone,0.040902,NaN,NaN
4,1,55,11,Alabama,Southside,33.904039,-86.053867,Ozone,0.044016,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1639,72,53,3,Puerto Rico,NaN,18.383983,-65.618888,NaN,NaN,PM2.5 - Local Conditions,6.868067
1640,72,57,12,Puerto Rico,Guayama,17.955378,-66.162122,NaN,NaN,PM2.5 - Local Conditions,5.303571
1641,72,61,5,Puerto Rico,Guaynabo,18.440095,-66.114460,NaN,NaN,PM2.5 - Local Conditions,7.487179
1642,72,113,4,Puerto Rico,Ponce,18.009558,-66.627249,NaN,NaN,PM2.5 - Local Conditions,6.285841


In [22]:
# Remove unneccesary columns from the Data frame
merge_df = merge_df[["state", "city", "latitude", "longitude", "ozone", "mean_ozone", "pm25", "mean_pm25"]]
merge_df.head()

,state,city,latitude,longitude,ozone,mean_ozone,pm25,mean_pm25
0,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.040489,PM2.5 - Local Conditions,7.551402
1,Alabama,Muscle Shoals,34.762619,-87.638097,Ozone,0.038221,PM2.5 - Local Conditions,7.213235
2,Alabama,Crossville,34.289001,-85.970065,Ozone,0.044582,NaN,NaN
3,Alabama,Wetumpka,32.535680,-86.255193,Ozone,0.040902,NaN,NaN
4,Alabama,Southside,33.904039,-86.053867,Ozone,0.044016,NaN,NaN


## Append method to put together two data frames

In [23]:
append_df = ozone.append(pm)
append_df.head(30)

,State Code,County Code,Site Num,state,city,latitude,longitude,pollutant,average
3,1,3,10,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.040489
15,1,33,1002,Alabama,Muscle Shoals,34.762619,-87.638097,Ozone,0.038221
27,1,49,9991,Alabama,Crossville,34.289001,-85.970065,Ozone,0.044582
31,1,51,4,Alabama,Wetumpka,32.535680,-86.255193,Ozone,0.040902
46,1,55,11,Alabama,Southside,33.904039,-86.053867,Ozone,0.044016
54,1,69,4,Alabama,Dothan,31.188933,-85.423094,Ozone,0.037248
80,1,73,23,Alabama,Birmingham,33.553056,-86.815000,Ozone,0.039671
248,1,73,1003,Alabama,Fairfield,33.485556,-86.915000,Ozone,0.044638
252,1,73,1005,Alabama,NaN,33.331111,-87.003611,Ozone,0.043936
272,1,73,1010,Alabama,Leeds,33.545278,-86.549167,Ozone,0.043135


## Make final version of ozone

In [24]:
ozone_final = ozone[["state", "city", "latitude", "longitude", "pollutant", "average"]]
ozone_final.head()

,state,city,latitude,longitude,pollutant,average
3,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.040489
15,Alabama,Muscle Shoals,34.762619,-87.638097,Ozone,0.038221
27,Alabama,Crossville,34.289001,-85.970065,Ozone,0.044582
31,Alabama,Wetumpka,32.535680,-86.255193,Ozone,0.040902
46,Alabama,Southside,33.904039,-86.053867,Ozone,0.044016


## Make final version of pm

In [25]:
pm_final = pm[["state", "city", "latitude", "longitude", "pollutant", "average"]]
pm_final.head()

,state,city,latitude,longitude,pollutant,average
7,Alabama,Fairhope,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402
11,Alabama,Ashland,33.284928,-85.803608,PM2.5 - Local Conditions,7.550467
19,Alabama,Muscle Shoals,34.762619,-87.638097,PM2.5 - Local Conditions,7.213235
23,Alabama,Crossville,34.288567,-85.969858,PM2.5 - Local Conditions,7.550000
42,Alabama,Gadsden,33.991494,-85.992647,PM2.5 - Local Conditions,8.410526


## CSV COPIES

In [ ]:
# csv files of each df.. just in case?
base_df.to_csv('baseDFcsv.csv', index=False, header=True)

In [ ]:
ozone_final.to_csv('ozoneDFcsv.csv', index=False, header=True)

In [ ]:
pm_final.to_csv('pmDFcsv.csv', index=False, header=True)

In [ ]:
merge_df.to_csv('merge2.csv', index=False, header=True)

In [ ]:
append_df.to_csv('append.csv', index=False, header=True)

## Make Json files

In [27]:
merge_df.to_json('merge2.json')

In [28]:
append_df.to_json('append.json')

## SQLITE DATABASE

In [ ]:
from sqlalchemy import create_engine

In [ ]:
# Connect to the database
engine = create_engine('sqlite:///aqi.sqlite', echo=False)
# I also created a db file:
# engine = create_engine('sqlite:///aqi.db', echo=False)
sqlite_connection = engine.connect()

In [ ]:
# Adding the base DF (I think this would give it its own table?)
base_df.to_sql('base_2', con=engine)

In [ ]:
# Adding the ozone DF
ozone_final.to_sql('ozone_final', con=engine)

In [ ]:
# Adding the pm DF
pm_final.to_sql('pm_final', con=engine)

In [ ]:
# Adding the merge2 DF
merge_df.to_sql('merge2', con=engine)

In [ ]:
# Adding the append DF
append_df.to_sql('append', con=engine)

In [ ]:
# Confirming tables are in the DB
engine.execute("SELECT * FROM base").fetchall()

In [ ]:
# ozone check
engine.execute("SELECT * FROM ozone").fetchall()

In [ ]:
# pm check
engine.execute("SELECT * FROM pm").fetchall()

In [ ]:
# merge2 check
engine.execute("SELECT * FROM merge2").fetchall()

In [ ]:
# append check
engine.execute("SELECT * FROM append").fetchall()

In [ ]:
# All the table names in the sqlite file (aqi.sqlite)
engine.table_names()